In [1]:
import pandas as pd
import numpy as np

In [66]:
ap_code = pd.read_csv('data/L_AIRPORT.csv')
ap_id = pd.read_csv('data/L_AIRPORT_ID.csv')
airports = pd.read_csv('data/airports.csv')

In [3]:
ap_code.shape, ap_id.shape

((6444, 2), (6428, 2))

In [67]:
ap_code.head()

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [64]:
ap_id.head()

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


In [65]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [68]:
#only keeping airport codes that are in the airports table
ap_code = ap_code.merge(airports[['IATA_CODE']],how='inner',left_on='Code',right_on='IATA_CODE')

In [69]:
ap_code.head()

,Code,Description,IATA_CODE
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",ABE
1,ABI,"Abilene, TX: Abilene Regional",ABI
2,ABQ,"Albuquerque, NM: Albuquerque International Sun...",ABQ
3,ABR,"Aberdeen, SD: Aberdeen Regional",ABR
4,ABY,"Albany, GA: Southwest Georgia Regional",ABY


In [70]:
#check if unique
ap_code.shape, len(ap_code['Description'].unique())

((322, 3), 322)

In [71]:
match = ap_code.merge(ap_id,how='inner',on='Description')

In [72]:
match.head()

,Code_x,Description,IATA_CODE,Code_y
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",ABE,10135
1,ABI,"Abilene, TX: Abilene Regional",ABI,10136
2,ABQ,"Albuquerque, NM: Albuquerque International Sun...",ABQ,10140
3,ABR,"Aberdeen, SD: Aberdeen Regional",ABR,10141
4,ABY,"Albany, GA: Southwest Georgia Regional",ABY,10146


In [73]:
#Read flight data for the year 2015
data = pd.read_csv(r'data/flights.csv',dtype={'ORIGIN_AIRPORT':str,'DESTINATION_AIRPORT':str})

In [74]:
#find the entries that need fixing
code_list = airports['IATA_CODE'].values
ind = np.where(~data['ORIGIN_AIRPORT'].isin(code_list))

In [75]:
ind[0].shape

(486165,)

In [77]:
data[['ORIGIN_AIRPORT']].iloc[ind].astype(int).merge(
    match,how='left',left_on='ORIGIN_AIRPORT',right_on='Code_y').head()

,ORIGIN_AIRPORT,Code_x,Description,IATA_CODE,Code_y
0,14747,SEA,"Seattle, WA: Seattle/Tacoma International",SEA,14747
1,14771,SFO,"San Francisco, CA: San Francisco International",SFO,14771
2,12889,LAS,"Las Vegas, NV: McCarran International",LAS,12889
3,12892,LAX,"Los Angeles, CA: Los Angeles International",LAX,12892
4,14771,SFO,"San Francisco, CA: San Francisco International",SFO,14771


In [78]:
new_code = data[['ORIGIN_AIRPORT']].iloc[ind].astype(int).merge(
    match,how='left',left_on='ORIGIN_AIRPORT',right_on='Code_y')['Code_x'].values.astype(str)

In [79]:
new_code.shape

(486165,)

In [80]:
new_code.dtype

dtype('S3')

In [81]:
#replace the original column with new values
new_code_column = data['ORIGIN_AIRPORT'].values
new_code_column[ind] = new_code
data['ORIGIN_AIRPORT'] = new_code_column

In [85]:
#do the same for destination airport
col = 'DESTINATION_AIRPORT'
ind = np.where(~data[col].isin(code_list))
new_code = data[[col]].iloc[ind].astype(int).merge(
    match,how='left',left_on=col,right_on='Code_y')['Code_x'].values.astype(str)
new_code_column = data[col].values
new_code_column[ind] = new_code
data[col] = new_code_column

In [87]:
#write the corrected dataframe to a csv file
data.to_csv('data/flights_corrected.csv')

In [86]:
#testing result
i = 4849664
data.iloc[i]

YEAR                     2015
MONTH                      10
DAY                        30
DAY_OF_WEEK                 5
AIRLINE                    OO
FLIGHT_NUMBER            5427
TAIL_NUMBER            N963SW
ORIGIN_AIRPORT            SMF
DESTINATION_AIRPORT       SFO
SCHEDULED_DEPARTURE      1045
DEPARTURE_TIME           1039
DEPARTURE_DELAY            -6
TAXI_OUT                   32
WHEELS_OFF               1111
SCHEDULED_TIME             52
ELAPSED_TIME               71
AIR_TIME                   35
DISTANCE                   86
WHEELS_ON                1146
TAXI_IN                     4
SCHEDULED_ARRIVAL        1137
ARRIVAL_TIME             1150
ARRIVAL_DELAY              13
DIVERTED                    0
CANCELLED                   0
CANCELLATION_REASON       NaN
AIR_SYSTEM_DELAY          NaN
SECURITY_DELAY            NaN
AIRLINE_DELAY             NaN
LATE_AIRCRAFT_DELAY       NaN
WEATHER_DELAY             NaN
Name: 4849664, dtype: object

In [84]:
new_code_column[i]

'SMF'